In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [19]:
class my_env(Env) : 
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-5,5)
        self.shower_time = 60

    def step(self, action):
        self.state += action -1;

        self.shower_time -= 1

        if self.shower_time == 0:
            done = True
        else:
            done = False

        if self.state >= 37 and self.state <= 40:
            reward = 1
        else:
            reward = -1

        self.state += random.randint(-1,1)

        return self.state, reward, done, {}
    def render(self, mode='human'):
        pass
    def reset(self):
        self.state = 38 + random.randint(-5,5)
        self.shower_time = 60

        return self.state

In [20]:
env = my_env()

/home/goyome/.local/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
runtime = 3
for i in range(runtime):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action = random.randint(0,2)
        env.render()
        state, reward, done, info = env.step(action)
        score += reward
    
    print("episode :", i, "score :", score)


episode : 0 score : 14
episode : 1 score : -58
episode : 2 score : 8


In [5]:
import numpy as np
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Dense
from keras.optimizers import Adam


2022-12-08 10:15:41.124612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 10:15:41.292654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 10:15:41.292673: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-08 10:15:42.034761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [6]:
states = env.observation_space.shape
actions = env.action_space.n

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation = 'relu', input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [13]:
model = build_model(states, actions)

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [15]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [16]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [24]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=20000, visualize=False, verbose=1)

2022-12-08 10:26:46.977862: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_5_3/bias/Assign' id:1500 op device:{requested: '', assigned: ''} def:{{{node dense_5_3/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_3/bias, dense_5_3/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 20000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 14:27 - reward: -1.0000

2022-12-08 10:26:47.184832: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_5/BiasAdd' id:145 op device:{requested: '', assigned: ''} def:{{{node dense_5/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_5/MatMul, dense_5/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2022-12-08 10:26:47.221173: W tensorflow/c/c_api.cc:291] Operation '{name:'count_10/Assign' id:1653 op device:{requested: '', assigned: ''} def:{{{node count_10/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](count_10, count_10/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running th

10000/10000 [==============================] - 47s 5ms/step - reward: 0.6234
166 episodes - episode_reward: 37.446 [-60.000, 58.000] - loss: 14.948 - mae: 28.425 - mean_q: 42.754

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 0.6506
done, took 94.508 seconds


In [29]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
print(scores.history['episode_reward'])

Testing for 5 episodes ...
Episode 1: reward: 58.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 58.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
[58.0, 60.0, 58.0, 60.0, 60.0]


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)
dqn.load_weights('dqn_weights.h5f')